In [ ]:
# part 1: perform EBM analysis on the DeepMIP data
# we perform analysis on common r180x90 grid for all models
#  interpolation has been done with the `regrid_deepmip_database_v1.0.sh`script

import numpy as np
import xarray as xr
import os

from deepmip_dict import deepmip_dict

